# 00 – Setup & Bootstrap (2025-08-09)

This notebook wires up **DI**, **logging**, **caching**, and a shared **configuration** so the rest of the notebooks
can reference the same services. It also includes a quick **Ollama** health check and a tiny **in‑memory API** for employees.

> Assumes you're running in VS Code + .NET Interactive. If not, you can still copy the C# code into your project.


In [3]:
#!csharp
#pragma warning disable 1701, 1702
#nullable enable
#load "SharedDefs.csx" // Load shared definitions and services
#r "nuget: Microsoft.Extensions.DependencyInjection, 8.0.0"
#r "nuget: Microsoft.Extensions.Logging.Console, 8.0.0"
#r "nuget: Microsoft.Extensions.Caching.Memory, 8.0.0"
#r "nuget: System.Text.Json, 8.0.4"
#r "nuget: RestSharp, 112.1.0"

using System;
using System.Threading;
using System.Threading.Tasks;
using System.Collections.Generic;
using System.Text.Json;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.Caching.Memory;
using RestSharp;



public record Slots(
    string[]? Names = null,
    DateTime? Date = null,
    (DateTime Start, DateTime End)? Range = null,
    string? Operator = null,
    string? Department = null,
    string? Role = null
);

public record QuerySpec(Intent Intent, Slots Slots);

public record Employee(
    string DisplayName,
    string Email,
    string Department,
    string Role,
    DateTime OriginalHireDate
);

public class AppConfig
{
    public string ApiBaseUrl { get; init; } = "http://localhost:4000"; // replace with your API later
    public string OllamaBaseUrl { get; init; } = "http://localhost:11434";
    public string OllamaModel { get; init; } = "phi3:mini";
    public TimeSpan CacheTtl { get; init; } = TimeSpan.FromMinutes(2);
}

// Service interfaces
public interface ISlotFiller { (Slots slots, double confidence, string? clarification) Fill(string query); }
public interface IQueryDispatcher { Task<object> ExecuteAsync(QuerySpec spec, CancellationToken ct = default); }
public interface IEmployeeApi { Task<IReadOnlyList<Employee>> GetEmployeesAsync(CancellationToken ct = default); }
public interface IAnswerFormatter {
    Task<(bool usedLlm, string text)> FormatAsync(QuerySpec spec, object rawData, CancellationToken ct = default);
}

public static class Services
{
    public static ServiceProvider Build()
    {
        var services = new ServiceCollection();

        services.AddLogging(b => b.AddSimpleConsole(o => { o.SingleLine = true; o.TimestampFormat = "HH:mm:ss "; }));
        services.AddMemoryCache();
        services.AddSingleton(new AppConfig());

        // Minimal in-memory API stub; replace with HttpClient-based implementation later.
        services.AddSingleton<IEmployeeApi, InMemoryEmployeeApi>();

        // Placeholders; real implementations are provided in other notebooks.
        services.AddSingleton<IIntentClassifier, DummyIntentClassifier>();
        services.AddSingleton<ISlotFiller, DummySlotFiller>();
        services.AddSingleton<IQueryDispatcher, DummyDispatcher>();
        services.AddSingleton<IAnswerFormatter, DummyFormatter>();

        return services.BuildServiceProvider();
    }
}

// Dummy implementations to be replaced later
public class DummyIntentClassifier : IIntentClassifier
{
    public (Intent intent, double confidence) Classify(string query) => (Intent.Unknown, 0.0);
}
public class DummySlotFiller : ISlotFiller
{
    public (Slots slots, double confidence, string? clarification) Fill(string query) => (new Slots(), 0.0, null);
}
public class DummyDispatcher : IQueryDispatcher
{
    public Task<object> ExecuteAsync(QuerySpec spec, CancellationToken ct = default) =>
        Task.FromResult<object>(Array.Empty<Employee>());
}
public class DummyFormatter : IAnswerFormatter
{
    public Task<(bool usedLlm, string text)> FormatAsync(QuerySpec spec, object rawData, CancellationToken ct = default) =>
        Task.FromResult((false, "Formatter not implemented yet."));
}

// In-memory API seed
public class InMemoryEmployeeApi : IEmployeeApi
{
    private readonly IMemoryCache _cache;
    private readonly AppConfig _cfg;

    public InMemoryEmployeeApi(IMemoryCache cache, AppConfig cfg)
    {
        _cache = cache;
        _cfg = cfg;
    }

    public Task<IReadOnlyList<Employee>> GetEmployeesAsync(CancellationToken ct = default)
    {
        return Task.FromResult((IReadOnlyList<Employee>)_cache.GetOrCreate("employees", e =>
        {
            e.AbsoluteExpirationRelativeToNow = _cfg.CacheTtl;
            return new List<Employee> {
                new("Rick Sanchez","rick.sanchez@company.com","Engineering","Staff Engineer", new DateTime(2015,5,10)),
                new("Morty Smith","morty.smith@company.com","Engineering","Engineer I", new DateTime(2022,9,14)),
                new("Summer Smith","summer.smith@company.com","Marketing","Manager", new DateTime(2019,3,6)),
                new("Beth Smith","beth.smith@company.com","People","Director", new DateTime(2012,11,2)),
                new("Jerry Smith","jerry.smith@company.com","Sales","Account Executive", new DateTime(2021,1,20)),
            };
        })!);
    }
}


Installed Packages Microsoft.Extensions.Caching.Memory, 8.0.0 Microsoft.Extensions.DependencyInjection, 8.0.0 Microsoft.Extensions.Logging.Console, 8.0.0 RestSharp, 112.1.0 System.Text.Json, 8.0.4

## Ollama Health Check

Quick ping to confirm the daemon and model are available.


In [4]:
var cfg = new AppConfig();
var client = new RestClient(cfg.OllamaBaseUrl);
var req = new RestRequest("/api/tags", Method.Get);
try
{
    var res = await client.ExecuteAsync(req);
    Console.WriteLine($"Ollama status: {(int)res.StatusCode}");
}
catch (Exception ex)
{
    Console.WriteLine("Ollama check failed: " + ex.Message);
}


Ollama status: 0
